In [1]:
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from statsmodels.tsa.statespace.varmax import VARMAX
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
stk_data = yf.download("RELIANCE.NS", start="2021-06-01", end="2022-05-01")
print(stk_data.head())

[*********************100%***********************]  1 of 1 completed

Price             Close         High         Low         Open      Volume
Ticker      RELIANCE.NS  RELIANCE.NS RELIANCE.NS  RELIANCE.NS RELIANCE.NS
Date                                                                     
2021-06-01   987.374817  1002.443324  977.177342   986.054586    28013832
2021-06-02  1002.101929  1005.857676  981.957442   981.957442    24630162
2021-06-03  1005.925903  1024.294928  999.757398  1011.092856    23974257
2021-06-04   997.208069  1009.021575  994.362780  1008.270425    14468646
2021-06-07  1014.006470  1020.653023  994.704227  1002.443367    16336573


In [3]:
dataset = pd.read_csv("reliance_data.csv", skiprows=1)
import pandas as pd
# Flatten column MultiIndex if exists
if isinstance(stk_data.columns, pd.MultiIndex):
    stk_data.columns = [col[0] for col in stk_data.columns]

# Check data
print("Data fetched successfully!")
print(stk_data.head())

Data fetched successfully!
                  Close         High         Low         Open    Volume
Date                                                                   
2021-06-01   987.374817  1002.443324  977.177342   986.054586  28013832
2021-06-02  1002.101929  1005.857676  981.957442   981.957442  24630162
2021-06-03  1005.925903  1024.294928  999.757398  1011.092856  23974257
2021-06-04   997.208069  1009.021575  994.362780  1008.270425  14468646
2021-06-07  1014.006470  1020.653023  994.704227  1002.443367  16336573


In [4]:
stk_data=stk_data[["Open","High","Low","Close"]]
stk_data

,Open,High,Low,Close
Date,,,,
2021-06-01,986.054586,1002.443324,977.177342,987.374817
2021-06-02,981.957442,1005.857676,981.957442,1002.101929
2021-06-03,1011.092856,1024.294928,999.757398,1005.925903
2021-06-04,1008.270425,1009.021575,994.362780,997.208069
2021-06-07,1002.443367,1020.653023,994.704227,1014.006470
...,...,...,...,...
2022-04-25,1252.808769,1260.117952,1216.833432,1231.155029
2022-04-26,1238.007307,1276.837847,1236.956647,1267.998169
2022-04-27,1258.953109,1291.913373,1258.587584,1269.231812


In [6]:
scaler = MinMaxScaler()
data_scaled = pd.DataFrame(scaler.fit_transform(stk_data), columns=stk_data.columns, index=stk_data.index)

In [7]:
train_size = int(len(data_scaled) * 0.8)
train = data_scaled.iloc[:train_size]
test = data_scaled.iloc[train_size:]

In [8]:
try:
    model = VARMAX(train, order=(1, 1))  # VARMA(1,1) style
    model_fit = model.fit(disp=False)
except Exception as e:
    print("Error fitting VARMAX(1,1):", e)
    model = VARMAX(train, order=(1, 0))  # fallback to VAR(1)
    model_fit = model.fit(disp=False)


In [9]:
start = len(train)
end = len(data_scaled) - 1
pred_scaled = model_fit.predict(start=start, end=end)